# MLOps Zoomcamp Week 1
## Written by Ryan Gallagher
Date: 21/5/22

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


## Homework for Week 1

In [9]:
# Q1 How many trips in jan
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
len(df)

1154112

In [3]:
# Q2
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df['duration'].mean()

19.1672240937939

In [4]:
# Data Prep - drop outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]
len(df)

1109826

In [5]:
# Q3 Missing values management - Get % of missing values
(df['PUlocationID'].isna().sum())/len(df)*100


83.52732770722618

In [25]:
# convert missing values to -1 in both columns
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

In [53]:
# Q4 One Hot Encoding
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].astype(str)

dv = DictVectorizer()

train_dicts = df[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_train.shape

(1154112, 525)

In [43]:
# Q5 - Use code chunk for lesson 

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [55]:
# Q5 Train Model
df_train = read_dataframe('./data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)


10.528519107211466

In [56]:
# Q5
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014283199640522